# CapEx Forecast Model

This notebook demonstrates a **driver-based CapEx forecasting model** using the sample input in `data/sample_capex_input.xlsx`.

It covers:
- Loading and exploring the CapEx dataset
- Computing **total CapEx** by scenario, asset type, and program
- Building a **quarterly CapEx spend curve** using order timing
- Deriving a simple **depreciation schedule**
- Constructing **project-level cash flows** and estimating **NPV/IRR** under simple assumptions


In [ ]:
# Imports & settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
%matplotlib inline


In [ ]:
# Load CapEx input data
capex_path = '../data/sample_capex_input.xlsx'
df = pd.read_excel(capex_path)
print(df.shape)
df.head()

In [ ]:
# Basic sanity checks
df.info()

In [ ]:
# Core derived metrics

# Total CapEx per line item
df['Total_Cost_USD'] = df['Quantity'] * df['Unit_Cost_USD']

# Helper to split a "YYYYQn" string into year and quarter
def split_quarter(q_str):
    try:
        year = int(q_str[:4])
        q = int(q_str[-1])
        return year, q
    except Exception:
        return np.nan, np.nan

for col in ['Order_Quarter', 'Need_Quarter', 'Ramp_Start_Quarter']:
    years, qs = zip(*df[col].apply(split_quarter))
    df[f'{col}_Year'] = years
    df[f'{col}_Num'] = qs

# A numeric index for quarters (useful for sorting) – e.g., 2025Q1 -> 2025*4 + 1
df['Order_Q_Index'] = df['Order_Quarter_Year'] * 4 + df['Order_Quarter_Num']
df['Ramp_Q_Index'] = df['Ramp_Start_Quarter_Year'] * 4 + df['Ramp_Start_Quarter_Num']

df[['Asset_ID', 'Scenario', 'Order_Quarter', 'Need_Quarter', 'Ramp_Start_Quarter', 'Total_Cost_USD']].head()

## 1. Summary views

High-level cuts of the CapEx portfolio by scenario, asset type, and program.


In [ ]:
# Total CapEx by scenario
capex_by_scenario = df.groupby('Scenario')['Total_Cost_USD'].sum().sort_values(ascending=False)
capex_by_scenario

In [ ]:
capex_by_scenario.plot(kind='bar')
plt.ylabel('Total CapEx (USD)')
plt.title('Total CapEx by Scenario')
plt.tight_layout()
plt.show()

In [ ]:
# Total CapEx by Asset Type
capex_by_type = df.groupby('Asset_Type')['Total_Cost_USD'].sum().sort_values(ascending=False)
capex_by_type

In [ ]:
capex_by_type.plot(kind='bar')
plt.ylabel('Total CapEx (USD)')
plt.title('Total CapEx by Asset Type')
plt.tight_layout()
plt.show()

In [ ]:
# Total CapEx by Program (Project_Code)
capex_by_program = df.groupby(['Project_Code', 'Program_Name'])['Total_Cost_USD'].sum().sort_values(ascending=False)
capex_by_program

## 2. Quarterly CapEx spend curve (cash-out timing)

For simplicity, we'll assume CapEx **cash outflow** occurs at the `Order_Quarter`.
We then aggregate those by quarter and scenario.


In [ ]:
# Build quarterly CapEx spend using Order_Quarter as the cash outlay timing

# Create a proper period index for the order quarter
df['Order_Period'] = df['Order_Quarter'].apply(lambda x: pd.Period(x, freq='Q'))

# Aggregate total CapEx by scenario and order quarter
quarterly_capex = (
    df.groupby(['Scenario', 'Order_Period'])['Total_Cost_USD']
      .sum()
      .reset_index()
      .sort_values(['Order_Period', 'Scenario'])
)
quarterly_capex.head()

In [ ]:
# Pivot for plotting: rows = quarter, columns = scenario
q_pivot = quarterly_capex.pivot(index='Order_Period', columns='Scenario', values='Total_Cost_USD').fillna(0)
q_pivot

In [ ]:
# Plot quarterly CapEx spend by scenario
q_pivot.plot(kind='bar', stacked=False)
plt.ylabel('Quarterly CapEx (USD)')
plt.title('Quarterly CapEx Spend by Scenario (Order-Based)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 3. Simple straight-line depreciation schedule

- We assume depreciation starts at `Ramp_Start_Quarter`.
- We use **straight-line depreciation** over `Depreciation_Years`.
- We compute **annual depreciation** (not quarterly, to keep it simple).


In [ ]:
def build_depreciation_schedule(df_input: pd.DataFrame) -> pd.DataFrame:
    """Build an annual straight-line depreciation schedule per asset line.
    Returns a DataFrame with columns: Project_Code, Asset_ID, Year, Annual_Depreciation_USD
    """
    records = []
    for _, row in df_input.iterrows():
        total_cost = row['Total_Cost_USD']
        dep_years = int(row['Depreciation_Years']) if not pd.isna(row['Depreciation_Years']) else 0
        ramp_year = int(row['Ramp_Start_Quarter_Year']) if not pd.isna(row['Ramp_Start_Quarter_Year']) else None
        
        if dep_years <= 0 or ramp_year is None:
            continue
        
        annual_dep = total_cost / dep_years
        for i in range(dep_years):
            year = ramp_year + i
            records.append({
                'Project_Code': row['Project_Code'],
                'Program_Name': row['Program_Name'],
                'Asset_ID': row['Asset_ID'],
                'Year': year,
                'Annual_Depreciation_USD': annual_dep
            })
    return pd.DataFrame(records)

dep_schedule = build_depreciation_schedule(df)
dep_schedule.head()

In [ ]:
# Aggregate annual depreciation by program
annual_dep_by_program = (
    dep_schedule
    .groupby(['Project_Code', 'Program_Name', 'Year'])['Annual_Depreciation_USD']
    .sum()
    .reset_index()
    .sort_values(['Project_Code', 'Year'])
)
annual_dep_by_program.head()

In [ ]:
# Example plot: annual depreciation for a selected program
example_project = annual_dep_by_program['Project_Code'].iloc[0]
print('Example project:', example_project)

proj_dep = annual_dep_by_program[annual_dep_by_program['Project_Code'] == example_project]
proj_dep.plot(x='Year', y='Annual_Depreciation_USD', kind='bar')
plt.ylabel('Annual Depreciation (USD)')
plt.title(f'Annual Depreciation – {example_project}')
plt.tight_layout()
plt.show()

## 4. Project-level cash flows, NPV & IRR (illustrative)

To build a simple financial view by project:

1. Treat **CapEx** as a negative cash flow in the **order year**.
2. Assume a simple annual **benefit** (e.g., cost savings or margin contribution) that is a multiple of annual depreciation.
3. Compute **NPV** and **IRR** of the resulting cash flow stream.

These are **illustrative** and depend heavily on your real business assumptions.


In [ ]:
def build_project_cashflows(df_input: pd.DataFrame,
                            project_code: str,
                            discount_rate: float = 0.10,
                            benefit_multiple: float = 1.3) -> dict:
    """Build annual cash flows for a project and compute NPV/IRR.
    
    Assumptions:
    - CapEx is a negative cash flow in the order year (sum of Total_Cost_USD).
    - Annual benefit (e.g., cost savings) starts in the ramp year and equals:
      (Total_Cost_USD / Depreciation_Years) * benefit_multiple.
    """
    proj_df = df_input[df_input['Project_Code'] == project_code].copy()
    if proj_df.empty:
        raise ValueError(f'No rows found for project {project_code}')
    
    # 1) CapEx outflows by order year
    proj_df['Order_Year'] = proj_df['Order_Quarter_Year']
    capex_by_year = proj_df.groupby('Order_Year')['Total_Cost_USD'].sum() * -1  # negative
    
    # 2) Annual benefits by year based on depreciation pattern
    dep_sched = build_depreciation_schedule(proj_df)
    # Convert depreciation into a proxy for annual benefit
    dep_sched['Annual_Benefit_USD'] = dep_sched['Annual_Depreciation_USD'] * benefit_multiple
    benefit_by_year = dep_sched.groupby('Year')['Annual_Benefit_USD'].sum()
    
    # 3) Combine into net cash flow by year
    all_years = sorted(set(capex_by_year.index).union(benefit_by_year.index))
    cf_by_year = {}
    for y in all_years:
        cf_capex = capex_by_year.get(y, 0.0)
        cf_benefit = benefit_by_year.get(y, 0.0)
        cf_by_year[y] = cf_capex + cf_benefit
    
    # Convert to ordered list of cash flows
    years_sorted = sorted(cf_by_year.keys())
    cashflows = [cf_by_year[y] for y in years_sorted]
    
    # 4) NPV and IRR
    npv = sum(cf / ((1 + discount_rate) ** (i)) for i, cf in enumerate(cashflows))
    try:
        irr = np.irr(cashflows)
    except Exception:
        irr = np.nan
    
    return {
        'project_code': project_code,
        'years': years_sorted,
        'cashflows': cashflows,
        'npv': npv,
        'irr': irr,
        'discount_rate': discount_rate,
        'benefit_multiple': benefit_multiple,
    }

# Example: compute NPV/IRR for one project
example_proj = df['Project_Code'].iloc[0]
proj_results = build_project_cashflows(df, example_proj, discount_rate=0.10, benefit_multiple=1.3)
proj_results

In [ ]:
# Pretty-print the project cash flows
for year, cf in zip(proj_results['years'], proj_results['cashflows']):
    print(f"Year {year}: {cf:,.0f} USD")

print("\nNPV @ {0:.0%}: {1:,.0f} USD".format(proj_results['discount_rate'], proj_results['npv']))
print("IRR: {0:.2%}".format(proj_results['irr']) if not np.isnan(proj_results['irr']) else "IRR: N/A")

### Next ideas / extensions

- Add separate **benefit assumptions** per project or asset type.
- Model **quarterly depreciation & benefits** instead of annual.
- Integrate with a **dashboard** (e.g., Power BI / Tableau) using the aggregated outputs.
- Export intermediate tables (quarterly CapEx, annual depreciation, project cash flows) as CSV for downstream tools.
